In [1]:
#Created 2021-09-08
#Copyright Spencer W. Leifeld

In [2]:
from pyspark.sql import SparkSession as Session
from pyspark import SparkConf as Conf
from pyspark import SparkContext as Context

In [3]:
import os
os.environ['SPARK_LOCAL_IP']='192.168.1.2'
os.environ['HADOOP_HOME']='/home/geno1664/Developments/Github_Samples/RDS-ENV/hadoop'
os.environ['LD_LIBRARY_PATH']='$LD_LIBRARY_PATH:$HADOOP_HOME/lib/native'
os.environ['PYSPARK_DRIVER_PYTHON']='jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS']='notebook'
os.environ['PYSPARK_PYTHON']='python3'
os.environ['PYARROW_IGNORE_TIMEZONE']='1'

In [4]:
configuration = Conf().setAppName('RDS_2').setMaster('spark://GenoMachine:7077')
configuration.set('spark.executor.memory','10G').set('spark.driver.memory', '2G').set('spark.cores.max', '8')

In [5]:
context = Context(conf=configuration)

21/09/09 11:18:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
session = Session(context)

In [7]:
from Functions.IO import CSV_File

In [8]:
csvDF = CSV_File(session, r'/home/geno1664/Developments/Github_Samples/RDS-ENV/Rural_Development_Study_No2/IO/Jobs.csv')
employmentByJob = csvDF.GetSparkDF().select('State', 'County', 'PctEmpAgriculture', 'PctEmpConstruction', 'PctEmpMining', 'PctEmpTrade', 'PctEmpTrans', \
    'PctEmpInformation', 'PctEmpFIRE', 'PctEmpServices', 'PctEmpGovt', 'PctEmpManufacturing')

Now Reading: /home/geno1664/Developments/Github_Samples/RDS-ENV/Rural_Development_Study_No2/IO/Jobs.csv


In [9]:
employmentByJob = employmentByJob.withColumnRenamed('PctEmpAgriculture', 'Farmers').withColumnRenamed('PctEmpConstruction', 'Builders').withColumnRenamed('PctEmpMining', 'Miners') \
    .withColumnRenamed('PctEmpTrade', 'Retail_Associates').withColumnRenamed('PctEmpFIRE', 'Businessmen').withColumnRenamed('PctEmpServices', 'Hospitality_Associates') \
        .withColumnRenamed('PctEmpGovt', 'Civil_Servants').withColumnRenamed('PctEmpManufacturing', 'Craftsmen').withColumnRenamed('PctEmpInformation', 'Technologists') \
            .withColumnRenamed('PctEmpTrans', 'Teamsters')

In [10]:
employmentByJob = employmentByJob.where(employmentByJob.State != 'US')
employmentByJob = employmentByJob.repartition('State')

In [11]:
employmentByJob.show()

+-----+--------------+-------------+------------+-------------+-----------------+-------------+-------------+-------------+----------------------+--------------+-------------+
|State|        County|      Farmers|    Builders|       Miners|Retail_Associates|    Teamsters|Technologists|  Businessmen|Hospitality_Associates|Civil_Servants|    Craftsmen|
+-----+--------------+-------------+------------+-------------+-----------------+-------------+-------------+-------------+----------------------+--------------+-------------+
|   AZ|       Arizona| 0.9066464622|6.9362734607|  0.461149062|    14.3824716721| 5.3067438219| 1.8067447802| 8.4040479669|         49.9525658823|  4.8267488815| 7.0166080102|
|   AZ|        Apache| 1.7880434783|9.0217391304| 1.0815217391|     9.6847826087| 7.6032608696| 0.6684782609| 3.1847826087|         53.5054347826| 11.4456521739| 2.0163043478|
|   AZ|       Cochise|  2.239488117|6.7664533821| 0.4981718464|    12.9547531993| 4.4881170018| 1.4008226691| 4.19332723

In [12]:
csvDF = CSV_File(session, r'/home/geno1664/Developments/Github_Samples/RDS-ENV/Rural_Development_Study_No2/IO/People.csv')
educationRate = csvDF.GetSparkDF().select('State', 'County', 'Ed1LessThanHSPct', 'Ed2HSDiplomaOnlyPct', 'Ed3SomeCollegePct', 'Ed4AssocDegreePct', 'Ed5CollegePlusPct')

Now Reading: /home/geno1664/Developments/Github_Samples/RDS-ENV/Rural_Development_Study_No2/IO/People.csv


In [13]:
educationRate = educationRate.withColumnRenamed('Ed1LessThanHSPct', 'Some_High_School').withColumnRenamed('Ed2HSDiplomaOnlyPct', 'High_School_Degree') \
    .withColumnRenamed('Ed3SomeCollegePct', 'Some_College').withColumnRenamed('Ed4AssocDegreePct', 'Associates_Degree').withColumnRenamed('Ed5CollegePlusPct', 'College_Graduate')

In [14]:
educationRate = educationRate.where(educationRate.State != 'US')
educationRate = educationRate.repartition('State')

In [15]:
educationRate.show()

+-----+--------------+----------------+------------------+-------------+-----------------+----------------+
|State|        County|Some_High_School|High_School_Degree| Some_College|Associates_Degree|College_Graduate|
+-----+--------------+----------------+------------------+-------------+-----------------+----------------+
|   AZ|       Arizona|   12.8607054321|     23.8588772353|25.1858413213|     8.6277705042|   29.4668055071|
|   AZ|        Apache|   19.5977663574|      32.238759483| 28.234221006|     7.6286457986|    12.300607355|
|   AZ|       Cochise|   12.6165291708|     24.1034242746|28.7691552847|    11.4013167654|   23.1095745045|
|   AZ|      Coconino|    9.4800192585|     22.0389985556|23.8854116514|     8.4352431391|   36.1603273953|
|   AZ|          Gila|   13.6740401729|     27.8540554284|30.8619374523|     8.8761759471|   18.7337909992|
|   AZ|        Graham|   14.7431030119|     28.7226862398|30.8529486206|    10.5289800051|   15.1522821227|
|   AZ|      Greenlee|   11.

In [16]:
from databricks import koalas as ks

In [17]:
employmentByJob = employmentByJob.to_koalas().melt(id_vars=['State', 'County'], var_name='Employment_Catagory', value_name='Employment_Rate').to_spark()
educationRate = educationRate.to_koalas().melt(id_vars=['State', 'County'], var_name='Education_Catagory', value_name='Education_Rate').to_spark()

In [18]:
mainDF = employmentByJob.join(educationRate, on=['State', 'County'], how='cross') \
    .fillna(0, subset=['Employment_Rate', 'Education_Rate']).fillna('NONE', subset=['Employment_Catagory', 'Education_Catagory'])

In [19]:
mainDF.show()

+-----+-------+-------------------+---------------+------------------+--------------+
|State| County|Employment_Catagory|Employment_Rate|Education_Catagory|Education_Rate|
+-----+-------+-------------------+---------------+------------------+--------------+
|   AZ|Arizona|            Farmers|   0.9066464622|  College_Graduate| 29.4668055071|
|   AZ|Arizona|            Farmers|   0.9066464622| Associates_Degree|  8.6277705042|
|   AZ|Arizona|            Farmers|   0.9066464622|      Some_College| 25.1858413213|
|   AZ|Arizona|            Farmers|   0.9066464622|High_School_Degree| 23.8588772353|
|   AZ|Arizona|            Farmers|   0.9066464622|  Some_High_School| 12.8607054321|
|   AZ|Arizona|           Builders|   6.9362734607|  College_Graduate| 29.4668055071|
|   AZ|Arizona|           Builders|   6.9362734607| Associates_Degree|  8.6277705042|
|   AZ|Arizona|           Builders|   6.9362734607|      Some_College| 25.1858413213|
|   AZ|Arizona|           Builders|   6.9362734607|Hig

In [20]:
mainDF = mainDF.to_koalas()
mainDF['Hueristic'] = (mainDF['Employment_Rate'] / 100) * (mainDF['Education_Rate'] / 100) * 100
mainDF = mainDF.to_spark().select('Employment_Catagory', 'Education_Catagory', 'Hueristic')

In [21]:
mainDF = mainDF.groupBy('Employment_Catagory').pivot('Education_Catagory').mean()

In [22]:
mainDF.show()

+--------------------+-------------------+------------------+------------------+-------------------+-------------------+
| Employment_Catagory|  Associates_Degree|  College_Graduate|High_School_Degree|       Some_College|   Some_High_School|
+--------------------+-------------------+------------------+------------------+-------------------+-------------------+
|      Civil_Servants| 0.4991104160220651|1.2303817172617337|1.9110977940676062| 1.2117440410274674| 0.7875654587994716|
|             Farmers|0.47202349285225276|0.9679238794088135|1.6797872795044708|  1.095120845562527| 0.6669638029817897|
|           Teamsters| 0.5024450736540681|1.1749227199195111|1.9199246920936035|  1.202032973463126| 0.7515646439894451|
|Hospitality_Assoc...|  3.935964409264519|10.041532500582806|14.451049048084114|  9.269000425159337|  5.665075456559447|
|           Craftsmen| 1.1229021642002426|2.4930672079499114| 4.326239662101073|  2.558292794172981| 1.6696027899509687|
|            Builders| 0.6699352

In [23]:
CSV_File.ExportCSV('/home/geno1664/Developments/Github_Samples/RDS-ENV/Rural_Development_Study_No2/IO/output.csv', mainDF)

21/09/09 11:18:56 WARN TaskSetManager: Lost task 59.0 in stage 51.0 (TID 2234) (192.168.1.2 executor 0): java.io.IOException: Mkdirs failed to create file:/home/geno1664/Developments/Github_Samples/RDS-ENV/Rural_Development_Study_No2/IO/output.csv/_temporary/0/_temporary/attempt_202109091118551324996356683812108_0051_m_000059_2234 (exists=false, cwd=file:/home/hadoop/.workspace/app-20210909111836-0003/0)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:458)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:443)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1118)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1098)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:987)
	at org.apache.spark.sql.execution.datasources.CodecStreams$.createOutputStream(CodecStreams.scala:81)
	at org.apache.spark.sql.execution.datasources.CodecStreams$.createOutputStreamWriter(CodecStreams.scala:92)
	at org.apache.spark.sql.e

Py4JJavaError: An error occurred while calling o882.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:188)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at org.apache.